In [2]:
!pip install optuna catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import optuna
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
os.chdir('/content/drive/MyDrive/Euron')

In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [7]:
#train.drop(columns=['ID', 'Book-ID', 'Year-Of-Publication'], inplace=True)
#test.drop(columns=['ID', 'Book-ID', 'Year-Of-Publication'], inplace=True)
# location 전처리
# Location 분리
train['Location'] = train['Location'].str.replace(r'[^0-9a-zA-Z:,]', '')
train['city'] = train['Location'].apply(lambda x: x.split(',')[0].strip())
train['state'] = train['Location'].apply(lambda x: x.split(',')[1].strip())
train['country'] = train['Location'].apply(lambda x: x.split(',')[2].strip())

train.drop('Location', axis=1, inplace=True)
train = train.replace('na', np.nan)
train = train.replace('', np.nan)

# Location 최빈값으로 대체
mode_city = train['city'].mode()[0]
mode_state = train['state'].mode()[0]
mode_country = train['country'].mode()[0]

# 결측값 대체
train['city'].fillna(mode_city, inplace=True)
train['state'].fillna(mode_state, inplace=True)
train['country'].fillna(mode_country, inplace=True)

<ipython-input-7-4af6f511e24b>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  train['Location'] = train['Location'].str.replace(r'[^0-9a-zA-Z:,]', '')


In [8]:
## Age 이상치 처리
# 이상치: 5세 이하 90세 초과
train.loc[(train['Age'] <= 5)|(train['Age'] > 90), 'Age'] = np.nan
train['Age'].isnull().sum() # 5세 이하 또는 90세 초과인 데이터 4497개
# 평균값으로 대체
train['Age'].mean() # Age (이상치 제거 후) 평균: 36.541634752034845
train['Age'] = train['Age'].fillna(train['Age'].mean())

## Age 범주화
# Age_group: 이상치 처리 & 5세 단위로 범주화
train['Age_group'] = pd.cut(train['Age'], bins = list(range(5, 91, 5)))

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder() #labelencoder함수를 가져온다.
encoder.fit(train['Age_group']) #내가 가지고 있는 데이터에 학습시킨다
digit_label = encoder.transform(train['Age_group'])
train['Age_group'] = digit_label # encoding 잘됐나?
#train['Age_group']

In [9]:
# 소문자 통일, 특수문자 제거 (종류: 15505 -> 15021)
train['Publisher'] = train['Publisher'].str.lower().replace('[^0-9a-zA-Z\s]', '', regex=True)

# 소문자 통일, 특수문자 제거, 띄어쓰기 제거 (종류: 15505 -> 14840)
train['Publisher'] = train['Publisher'].str.lower().replace('[^0-9a-zA-Z]', '', regex=True)

In [10]:
train['Book-Author'] = train['Book-Author'].str.lower().replace('[^0-9a-zA-Z]', '', regex=True)

In [11]:
# 출판년도 그룹화
# 만약 출판연도가 null이 있다면 정보없음(-1)로 채움
# train['Year-Of-Publication'] = train['Year-Of-Publication'].fillna(-1)

labels = ['Unknown', '-1800', '1800-1850', '1850-1900', '1900-1910', '1910-1920', '1920-1930', '1930-1940', '1940-1950','1950-1960', '1960-1970', '1970-1980', '1980-1990', '1990-2000', '2000-2010', '2010-2020', '2020-']
bins = [-1, 0, 1800, 1850, 1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2020, 3000]
train['Pub_gb'] = pd.cut(train['Year-Of-Publication'], bins, labels = labels,include_lowest = True)
train = train.drop(columns =['Year-Of-Publication'])
#train['Year-Of-Publication'] = pd.cut(train['Year-Of-Publication'], bins=bins, labels=labels)


encoder2 = LabelEncoder() #labelencoder함수를 가져온다.
encoder2.fit(train['Pub_gb']) #내가 가지고 있는 데이터에 학습시킨다
digit_label = encoder2.transform(train['Pub_gb'])
train['Pub_gb'] = digit_label # encoding 잘됐나?
#train['Pub_gb']

In [13]:
# Title 데이터 정리
import re # 정규표현 모듈

# 정규표현식으로 대소문자 통일, 특수문자 제거
train['Book-Title'] = [re.sub(r'[^0-9a-zA-Z:,]',  ' ',str(i)) for i in train['Book-Title']]
train['Main_Title'] = [i.split('  ')[0] for i in train['Book-Title']]
train['Sub_Title'] = [''.join(i.split('  ')[1:]) for i in train['Book-Title']]
train['Sub_Title'] = np.where(train['Sub_Title'] == '', 'No_SUB', train['Sub_Title'])

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 871393 entries, 0 to 871392
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ID           871393 non-null  object 
 1   User-ID      871393 non-null  object 
 2   Book-ID      871393 non-null  object 
 3   Book-Rating  871393 non-null  int64  
 4   Age          871393 non-null  float64
 5   Book-Title   871393 non-null  object 
 6   Book-Author  871393 non-null  object 
 7   Publisher    871393 non-null  object 
 8   city         871393 non-null  object 
 9   state        871393 non-null  object 
 10  country      871393 non-null  object 
 11  Age_group    871393 non-null  int64  
 12  Pub_gb       871393 non-null  int64  
 13  Main_Title   871393 non-null  object 
 14  Sub_Title    871393 non-null  object 
dtypes: float64(1), int64(3), object(11)
memory usage: 99.7+ MB


In [16]:
y = train['Book-Rating']

X = train.drop(['Book-Rating','Book-ID','Book-Title','ID'], axis=1)

In [17]:
kf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

cat_feature = ["User-ID",  "Age_group","Pub_gb","Publisher","country","Book-Author","state","city","Main_Title","Sub_Title"]

In [19]:
def objective_cat(trial):
    """
    Objective function to tune a `CatBoostRegressor` model.
    """

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

    params = {
        'objective' :  'RMSE',
        'n_estimators':trial.suggest_int("n_estimators", 30, 10000),
        'od_wait':trial.suggest_int('od_wait', 500, 2300),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'depth': trial.suggest_int('depth',1, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
        'random_state' : 42,
        'task_type' : "GPU",
        'verbose' : False
    }


    model = CatBoostRegressor(
        **params,
        cat_features=["User-ID",  "Age_group","Pub_gb","Publisher","country","Book-Author","state","city","Main_Title","Sub_Title"]
    )
    model.fit(x_train, y_train, eval_set = [(x_test, y_test)], early_stopping_rounds = 500, verbose = 2000)

    pred = model.predict(x_test)

    mae = mean_squared_error(y_test, pred, squared=False)


    return mae

In [20]:
study = optuna.create_study(direction="minimize")
study.optimize(objective_cat, n_trials=10)

[I 2024-02-06 10:23:44,933] A new study created in memory with name: no-name-5041b4b1-df8d-42f8-8b68-6cfb0928f644
<ipython-input-19-88818f8e1d71>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
<ipython-input-19-88818f8e1d71>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
<ipython-input-19-88818f8e1d71>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'random_strength': trial.suggest_uniform('random_strength',10,50),


0:	learn: 3.7491053	test: 3.7427320	best: 3.7427320 (0)	total: 127ms	remaining: 1m 26s
681:	learn: 3.2666421	test: 3.2517293	best: 3.2516922 (659)	total: 1m 17s	remaining: 0us
bestTest = 3.251692238
bestIteration = 659
Shrink model to first 660 iterations.


[I 2024-02-06 10:25:28,408] Trial 0 finished with value: 3.2516924284683286 and parameters: {'n_estimators': 682, 'od_wait': 886, 'learning_rate': 0.16168595684152917, 'reg_lambda': 58.252387412785495, 'random_strength': 38.20371032813519, 'depth': 8, 'min_data_in_leaf': 4}. Best is trial 0 with value: 3.2516924284683286.
<ipython-input-19-88818f8e1d71>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
<ipython-input-19-88818f8e1d71>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
<ipython-input-19-88818f8e1d71>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. Th

0:	learn: 3.5333517	test: 3.4989478	best: 3.4989478 (0)	total: 57.2ms	remaining: 2m 43s
bestTest = 3.266890288
bestIteration = 501
Shrink model to first 502 iterations.


[I 2024-02-06 10:26:35,529] Trial 1 finished with value: 3.2668905786490066 and parameters: {'n_estimators': 2863, 'od_wait': 1214, 'learning_rate': 0.9258427148178731, 'reg_lambda': 46.77590350966542, 'random_strength': 10.755186231674942, 'depth': 3, 'min_data_in_leaf': 11}. Best is trial 0 with value: 3.2516924284683286.
<ipython-input-19-88818f8e1d71>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
<ipython-input-19-88818f8e1d71>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
<ipython-input-19-88818f8e1d71>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. 

...

[I 2024-02-06 10:41:23,973] Trial 7 finished with value: 3.262159528534828 and parameters: {'n_estimators': 4368, 'od_wait': 1748, 'learning_rate': 0.27573946067279853, 'reg_lambda': 57.626942204764475, 'random_strength': 12.713945216803388, 'depth': 14, 'min_data_in_leaf': 4}. Best is trial 4 with value: 3.249889191106378.
<ipython-input-19-88818f8e1d71>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
<ipython-input-19-88818f8e1d71>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
<ipython-input-19-88818f8e1d71>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. 

0:	learn: 3.5811853	test: 3.5569345	best: 3.5569345 (0)	total: 34ms	remaining: 4m 14s
bestTest = 3.264751088
bestIteration = 1499
Shrink model to first 1500 iterations.


[I 2024-02-06 10:42:41,727] Trial 8 finished with value: 3.2647513701226143 and parameters: {'n_estimators': 7486, 'od_wait': 897, 'learning_rate': 0.6792514070680401, 'reg_lambda': 95.722028431747, 'random_strength': 24.029269659067317, 'depth': 2, 'min_data_in_leaf': 30}. Best is trial 4 with value: 3.249889191106378.
<ipython-input-19-88818f8e1d71>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
<ipython-input-19-88818f8e1d71>:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
<ipython-input-19-88818f8e1d71>:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This

0:	learn: 3.4679572	test: 3.4289433	best: 3.4289433 (0)	total: 620ms	remaining: 54m 11s
bestTest = 3.30164971
bestIteration = 16
Shrink model to first 17 iterations.


[I 2024-02-06 10:48:17,311] Trial 9 finished with value: 3.3016501774030567 and parameters: {'n_estimators': 5246, 'od_wait': 2294, 'learning_rate': 0.9534697231454429, 'reg_lambda': 67.37683210145836, 'random_strength': 13.102977553645925, 'depth': 15, 'min_data_in_leaf': 11}. Best is trial 4 with value: 3.249889191106378.


In [21]:
params=study.best_params
params['objective'] = 'RMSE'
params['random_state'] = 42
params['verbose'] = False
params['task_type'] = "GPU"
params['cat_features'] = ["User-ID",  "Age_group","Pub_gb","Publisher","country","Book-Author","state","city","Main_Title","Sub_Title"]

In [22]:
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

Best Score: 3.249889191106378
Best trial: {'n_estimators': 5411, 'od_wait': 1699, 'learning_rate': 0.07706713006326803, 'reg_lambda': 85.29820422306061, 'random_strength': 44.314299423505496, 'depth': 12, 'min_data_in_leaf': 24}


In [23]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [24]:
# 하이퍼파라미터 중요도
optuna.visualization.plot_param_importances(study)